In [1]:
import geopandas as gpd
import psycopg2
import folium
import fiona

In [2]:
rapperswil_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1683921"
zurich_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682248"
switzerland_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -51701"
hombrechtikon_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682143"
staefa_polygon_query = "SELECT way FROM planet_osm_polygon WHERE osm_id = -1682214"

rapperswil_location = [47.226, 8.818]
zurich_location = [47.3763, 8.5403]
hombrechtikon_location = [47.25212, 8.77038]
staefa_location = [47.24062, 8.72303]

epsg = 3857

polygon_query = rapperswil_polygon_query
location = zurich_location

In [3]:
shop_tags = ['mall', 'bakery', 'beverages', 'butcher', 'chocolate', 'coffee',
'confectionery', 'deli', 'frozen_food', 'greengrocer', 'healthfood',
'ice_cream', 'pasta', 'pastry', 'seafood', 'spices', 'tea', 'department_store',
'supermarket', 'bag', 'boutique', 'clothes', 'fashion', 'jewelry', 'leather',
'shoes', 'tailor', 'watches', 'chemist', 'cosmetics', 'hairdresser',
'medical_supply', 'electrical', 'hardware', 'electronics', 'sports',
'swimming_pool', 'collector', 'games', 'music', 'books', 'gift', 'stationery',
'ticket', 'laundry', 'pet', 'tobacco', 'toys']

amenity_tags = ['pub', 'bar', 'cafe', 'restaurant', 'pharmacy', 'bank', 'fast_food',
'food_court', 'ice_cream', 'library', 'ferry_terminal', 'clinic', 'doctors', 'hospital',
'pharmacy', 'veterinary', 'dentist', 'arts_centre', 'cinema',
'community_centre', 'casino', 'fountain', 'nightclub', 'studio', 'theatre',
'dojo', 'internet_cafe', 'marketplace', 'post_opffice', 'townhall']

leisure_tags = ['adult_gaming_centre', 'amusement_arcade', 'beach_resort',
'fitness_centre', 'garden', 'ice_rink', 'sports_centre', 'water_park']

landuse_tags = ['retail']

tags = {
    'shop_tags': shop_tags,
    'amenity_tags': amenity_tags,
    'leisure_tags': leisure_tags,
    'landuse_tags': landuse_tags
}

In [4]:
poi_query = """
(SELECT way AS geometry FROM planet_osm_polygon
    WHERE (amenity = ANY(ARRAY{amenity_tags})
            OR shop = ANY(ARRAY{shop_tags})
            OR leisure = ANY(ARRAY{leisure_tags})
            OR landuse = ANY(ARRAY{landuse_tags}))
            
           AND access IS DISTINCT FROM 'private'
           AND st_within(way, ({polygon})))
           
UNION ALL

(SELECT polygon.way AS geometry FROM planet_osm_polygon AS polygon
    INNER JOIN planet_osm_point AS point
        ON st_within(point.way, polygon.way)
    WHERE (point.amenity = ANY(ARRAY{amenity_tags})
            OR point.shop = ANY(ARRAY{shop_tags})
            OR point.leisure = ANY(ARRAY{leisure_tags})
            OR point.landuse = ANY(ARRAY{landuse_tags}))
            
        AND point.access IS DISTINCT FROM 'private'
        AND st_within(point.way, ({polygon}))
        AND polygon.building IS NOT NULL)
        
""".format(polygon=polygon_query, **tags) 

In [5]:
cluster_query = """
SELECT st_concavehull(cluster, 0.99) AS geometry FROM
(
WITH pois AS ({poi_query})
SELECT unnest(ST_ClusterWithin(geometry, 100)) AS cluster FROM pois
) AS clusters
""".format(poi_query=poi_query)

In [13]:
postgres_preclusters_query ="""
SELECT hull FROM preclusters;
"""

with psycopg2.connect("") as conn:
    clusters = gpd.read_postgis(postgres_preclusters_query, conn, geom_col='hull')

clusters.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=[47.3763, 8.5403], zoom_start=16)

folium.GeoJson(clusters).add_to(m)

m

In [19]:
geospark_cluster_query = """
WITH spark_clusters AS (
    SELECT ST_Transform(ST_ConvexHull(ST_Union(ST_SetSRID(ST_MakePoint(y, x), 4326))), 3857) AS geometry, cluster_id
    FROM geospark_preclustered_pois
    GROUP BY cluster_id
)
SELECT ST_ConvexHull(ST_Union(pois.geometry)) AS geometry FROM pois, spark_clusters
WHERE ST_Intersects(pois.geometry, spark_clusters.geometry)
GROUP BY cluster_id
"""

with psycopg2.connect("") as conn:
    clusters = gpd.read_postgis(geospark_cluster_query, conn, geom_col='geometry')

clusters.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=[47.3763, 8.5403], zoom_start=16)

folium.GeoJson(clusters).add_to(m)

m